# Final Project: Kickstarter Dataset
## Corbin Higgs, Zach Meyer, Jacob Ulman
##                              What makes a Kickstarter campaign succeed?

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score,  classification_report, confusion_matrix

%matplotlib inline

In [2]:
df = pd.read_csv("data/kickstarter.csv")
#df.head(2)

In [3]:
df['deadline'] = pd.to_datetime(df['deadline'])
df['launched'] = pd.to_datetime(df['launched'])

df['timediffdays'] = (df['deadline'] - df['launched']).dt.days

#df.head(2)

In [4]:
le = LabelEncoder()
df['maincat'] = le.fit_transform(df['main_category'])
df['state_encoded'] = le.fit_transform(df['state'])

df.head(2)

#Clearing out unkown, suspended, and live projects as they are not failed or successful
df = df.loc[(df['state_encoded'] != 5)]
df = df.loc[(df['state_encoded'] != 4)]
df = df.loc[(df['state_encoded'] != 2)]

#Re-encoding the states
df['state_encoded'] = le.fit_transform(df['state'])

#fixing encoding
state = []
for item, frame in df['state'].iteritems():
        if (frame == 'successful'):
            state += [1]
        else:
            state += [0]

df['state_encoded'] = state

In [5]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,ID,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real,timediffdays,maincat,state_encoded
ID,1,0.0017,0.00057,0.00064,-0.00047,6.7e-05,0.0019,-7.9e-05,-0.00096,-0.00089
goal,0.0017,1,0.0075,0.0042,0.0058,0.0053,0.94,0.0047,0.0037,-0.025
pledged,0.00057,0.0075,1,0.72,0.86,0.95,0.0053,0.00097,0.005,0.11
backers,0.00064,0.0042,0.72,1,0.7,0.75,0.0047,-0.00082,0.00045,0.13
usd pledged,-0.00047,0.0058,0.86,0.7,1,0.91,0.0065,0.00096,0.0058,0.098
usd_pledged_real,6.7e-05,0.0053,0.95,0.75,0.91,1,0.0058,0.0011,0.0062,0.11
usd_goal_real,0.0019,0.94,0.0053,0.0047,0.0065,0.0058,1,0.0046,0.002,-0.024
timediffdays,-7.9e-05,0.0047,0.00097,-0.00082,0.00096,0.0011,0.0046,1,0.0069,-0.028
maincat,-0.00096,0.0037,0.005,0.00045,0.0058,0.0062,0.002,0.0069,1,-0.032
state_encoded,-0.00089,-0.025,0.11,0.13,0.098,0.11,-0.024,-0.028,-0.032,1


In [6]:
feature_names = ['usd_goal_real','timediffdays','maincat','backers']
features = df[feature_names]
labels = df['state_encoded']

#data splitting
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=42)

In [11]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier(n_estimators=10)
scoring = 'accuracy'
# estimates accuracy
scores = cross_validate(model, x_train, y_train, scoring=[scoring] , cv=5)
best_score = np.max(scores['test_' + scoring])
print('Random Tree Best Classifier')
print('best {0} score : {1}'.format(scoring, best_score))
# create predictions 
y_pred = cross_val_predict(model, x_train, y_train, cv = 5)
print('true', confusion_matrix(y_train, y_pred), '\n predicted')

# Number of trees in random forest
n_estimators  = np.arange(10,30, 10)
# Number of features to consider at every split
# Maximum number of levels in tree
max_depth = [3,4,5]
# Minimum number of samples required to split a node
min_samples_split = np.arange(2, 6, 2)
# Minimum number of samples required at each leaf node
min_samples_leaf = np.arange(2, 6,2)
# Method of selecting samples for training each tree
param_grid= {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(param_grid)

model = RandomForestClassifier(n_estimators=10)
model = GridSearchCV(model, param_grid = param_grid,cv = 3,scoring=scoring, n_jobs = -1,  refit=True)

model = model.fit(x_train, y_train)


Random Tree Best Classifier
best accuracy score : 0.9210950404899078
true [[149021   9467]
 [ 10302  79414]] 
 predicted
{'n_estimators': array([10, 20]), 'max_depth': [3, 4, 5], 'min_samples_split': array([2, 4]), 'min_samples_leaf': array([2, 4])}


In [12]:


## Predict Data in the Test Set
pred_test = model.predict_proba(x_test)[:, 1]
pred_labels_test = model.predict(x_test)
print('predicted probablities: {}'.format(pred_test[0:5]))
print('predicted labeles: {}'.format(pred_labels_test[0:5]) )

# roc Auc score 
test_auc = roc_auc_score(y_test, pred_test)
print('test AUC {}'.format(test_auc))

# classifcation Report 
report = classification_report(y_test, pred_labels_test)
print(report)

# confusion Matrix
cm = confusion_matrix(y_test, pred_labels_test)

print('TN, FP \n FN TP')
print(cm)

predicted probablities: [0.01662273 0.01086001 0.38204365 0.01274082 0.00653988]
predicted labeles: [0 0 0 0 0]
test AUC 0.9734111146146005
              precision    recall  f1-score   support

           0       0.95      0.92      0.94     78010
           1       0.87      0.92      0.90     44240

    accuracy                           0.92    122250
   macro avg       0.91      0.92      0.92    122250
weighted avg       0.93      0.92      0.92    122250

TN, FP 
 FN TP
[[72137  5873]
 [ 3482 40758]]
